# - Data Warehouse -

### Het samenvoegen van bestaande tabellen voor het uiteindelijke 'Datawarehouse'

##### Importeren van benodigde dependencies

In [2]:
import pandas as pd
import pyodbc

import warnings
warnings.filterwarnings("ignore")

# Importeren van de create_connection en run_query functies uit de database_utils.py file
from utils.database_utils import create_connection, run_query

##### 1 - Opbouwen van dataframes voor elke database-tabel

In [3]:
aenc_customer_df = run_query("SELECT * FROM customer", "AenC")

nw_customer_df = run_query("SELECT * FROM Customers", "Northwind")

Je kunt vergelijkbare queries uitvoeren voor andere tabellen die je wilt opnemen.

#### 2 - Data transformatie

Na het opbouwen van de dataframes voor elke database-tabel, kunnen we beginnen met het transformeren van de data. Dit omvat het samenvoegen van tabellen, het toepassen van filters, het uitvoeren van berekeningen, enzovoort.

In [8]:
create_table_query = """
CREATE TABLE CustomerData (
    IDSK INT PRIMARY KEY,
    id VARCHAR(255),
    fname VARCHAR(255),
    lname VARCHAR(255),
    address VARCHAR(255),
    city VARCHAR(255),
    state VARCHAR(255),
    zip VARCHAR(20),
    phone VARCHAR(20),
    company_name VARCHAR(255),
    CompanyName VARCHAR(255),
    ContactName VARCHAR(255),
    ContactTitle VARCHAR(255),
    Region VARCHAR(255),
    PostalCode VARCHAR(20),
    Country VARCHAR(255),
    Fax VARCHAR(20)
);

"""

# Create the table in SQL Server
db_name = "test"
conn, cursor = create_connection(db_name)
cursor.execute(create_table_query)
conn.commit()
conn.close()

In [4]:
import pandas as pd

# Haal de klantgegevens op
aenc_customer_df = run_query("SELECT * FROM customer", "AenC")
nw_customer_df = run_query("SELECT * FROM Customers", "Northwind")

# Voeg de klantgegevens samen met een nieuwe primaire sleutel
aenc_customer_df['IDSK'] = range(1, len(aenc_customer_df) + 1)
nw_customer_df['IDSK'] = range(1, len(nw_customer_df) + 1)

# Merge de klantgegevens
merged_customer_df = pd.merge(aenc_customer_df, nw_customer_df, on='IDSK', how='inner')

# Toon het samengevoegde dataframe
merged_customer_df


,id,fname,lname,address,city,state,zip,phone,company_name,IDSK,...,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,101,Michaels,Devlin,3114 Pioneer Avenue,Rutherford,NJ,07070,2015558966,The Power Group,1,...,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
1,102,Beth,Reiser,1033 Whippany Road,New York,NY,10154,2125558725,AMF Corp.,2,...,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
2,103,Erin,Niedringhaus,1990 Windsor Street,Paoli,PA,19301,2155556513,Darling Associates,3,...,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
3,104,Meghan,Mason,550 Dundas Street East,Knoxville,TN,37919,6155555463,P.S.C.,4,...,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
4,105,Laura,McCarthy,1210 Highway 36,Carmel,IN,46032,3175558437,Amo & Sons,5,...,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,187,Sebouh,Bensoul,8024 Van Ness Way,Peoria,IL,61614,3095556915,Bensoul's Boutique,87,...,Wartian Herkku,Pirkko Koskitalo,Accounting Manager,Torikatu 38,Oulu,None,90110,Finland,981-443655,981-443655
87,188,Vartan,Berenberg,1020 110th Avenue N.E.,Northbrook,IL,60062,7085552914,Diva's Design,88,...,Wellington Importadora,Paula Parente,Sales Manager,"Rua do Mercado, 12",Resende,SP,08737-363,Brazil,(14) 555-8122,None
88,189,Herbert,Berejiklian,1535 Municiple Drive,Cambridge,MA,02142,6175553547,Out of Town Sports,89,...,White Clover Markets,Karl Jablonski,Owner,305 - 14th Ave. S. Suite 3B,Seattle,WA,98128,USA,(206) 555-4112,(206) 555-4115
89,190,Randy,Arlington,1635 N Franklin Turnpike,Detriot,MI,48214,3135555716,Jim Dandy's,90,...,Wilman Kala,Matti Karttunen,Owner/Marketing Assistant,Keskuskatu 45,Helsinki,None,21240,Finland,90-224 8858,90-224 8858


#### 3 - Data loading

Na het transformeren van de data, kunnen we de resulterende dataframe in de doeltabel van ons datawarehouse laden.

In [11]:
# Verbinding maken met de database
test_database_name = 'test'
conn, cursor = create_connection(test_database_name)

for index, row in merged_customer_df.iterrows():
    # Opstellen van de SQL-invoegquery met parameters
    query = """
        INSERT INTO CustomerData (
            IDSK,
            id,
            fname,
            lname,
            address,
            city,
            state,
            zip,
            phone,
            company_name,
            CompanyName,
            ContactName,
            ContactTitle,
            Region,
            PostalCode,
            Country,
            Fax
        ) 
        VALUES (
            ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
        )
    """
    # Parameters voor de query
    params = (
        row['IDSK'],
        row['id'],
        row['fname'],
        row['lname'],
        row['address'],
        row['city'],
        row['state'],
        row['zip'],
        row['phone'],
        row['company_name'],
        row['CompanyName'],
        row['ContactName'],
        row['ContactTitle'],
        row['Region'],
        row['PostalCode'],
        row['Country'],
        row['Fax']
    )
    # Uitvoeren van de query met parameters
    cursor.execute(query, params)

conn.commit()
conn.close()


**Note:** Voeg indien nodig zoveel Markdown- of codeblokken toe als nodig is.

#### 4 -  Data Quality Checks

Voeg controles toe om de kwaliteit van de gegevens te waarborgen voordat ze worden geladen in het datawarehouse:

In [12]:
# Controleren op ontbrekende waarden
missing_values = merged_customer_df.isnull().sum()

# Controleren op duplicaten
duplicate_rows = merged_customer_df.duplicated().sum()

# Weergave van resultaten
print("Aantal ontbrekende waarden:", missing_values)
print("Aantal duplicaten:", duplicate_rows)

Aantal ontbrekende waarden: id               0
fname            0
lname            0
address          0
city             0
state            0
zip              0
phone            0
company_name     0
IDSK             0
CustomerID       0
CompanyName      0
ContactName      0
ContactTitle     0
Address          0
City             0
Region          60
PostalCode       1
Country          0
Phone            0
Fax             22
dtype: int64
Aantal duplicaten: 0


**Note:** Dit is optioneel, het leek mij opzich best handig om te doen.

In [5]:
merged_customer_df.to_csv('merged_customer_df.csv', index=False)